# Google Drive to Git Repository Downloader

This notebook will:
1. Download files from a Google Drive folder
2. Clone your Git repository
3. Copy the files to the repository
4. Commit and push changes to GitHub

## Instructions:
1. Edit the configuration in Cell 2
2. Get a GitHub Personal Access Token from: https://github.com/settings/tokens
3. Run each cell in order (Shift+Enter)

---

In [ ]:
# CELL 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')
print("✓ Google Drive mounted successfully!")

In [ ]:
# CELL 2: Configuration - EDIT THESE VALUES

# Google Drive folder URL (public link)
GDRIVE_FOLDER_URL = "https://drive.google.com/drive/folders/15qe0A89GbjajpVfxx--ybPJ9AupIKDYK"

# Your GitHub repository URL
GIT_REPO_URL = "https://github.com/Encryptioner/acmp-4.0-for-engineers.git"

# Branch to push to
GIT_BRANCH = "claude/load-add-files-0178hz39eJYYhv1CW4QgRdQd"

# Folder name in the repo where files will be placed
TARGET_FOLDER = "gdrive_files"

# Working directories (usually don't need to change)
DOWNLOAD_FOLDER = "/content/downloads"
REPO_FOLDER = "/content/repo"

print("Configuration:")
print(f"  Google Drive Folder: {GDRIVE_FOLDER_URL}")
print(f"  Git Repository: {GIT_REPO_URL}")
print(f"  Git Branch: {GIT_BRANCH}")
print(f"  Target Folder: {TARGET_FOLDER}")

In [ ]:
# CELL 3: Install required packages
!pip install -q gdown
import os
import shutil
import re
from pathlib import Path

print("✓ Packages installed")

In [ ]:
# CELL 4: Download from Google Drive

# Extract folder ID
folder_match = re.search(r'/folders/([a-zA-Z0-9_-]+)', GDRIVE_FOLDER_URL)
if folder_match:
    folder_id = folder_match.group(1)
    print(f"Folder ID: {folder_id}")
else:
    print("Error: Could not extract folder ID")
    raise ValueError("Invalid Google Drive URL")

# Create download directory
os.makedirs(DOWNLOAD_FOLDER, exist_ok=True)

# Download the folder
print(f"\nDownloading files to {DOWNLOAD_FOLDER}...")
!gdown --folder {folder_id} -O {DOWNLOAD_FOLDER}

# List downloaded files
print("\n✓ Downloaded files:")
for root, dirs, files in os.walk(DOWNLOAD_FOLDER):
    for file in files:
        file_path = os.path.join(root, file)
        relative_path = os.path.relpath(file_path, DOWNLOAD_FOLDER)
        size = os.path.getsize(file_path)
        print(f"  {relative_path} ({size:,} bytes)")

In [ ]:
# CELL 5: Setup Git Authentication

from getpass import getpass

print("Git Authentication Setup")
print("=" * 60)
print("You need a GitHub Personal Access Token to push to GitHub.")
print("")
print("Get your token from: https://github.com/settings/tokens")
print("  1. Click 'Generate new token (classic)'")
print("  2. Give it a name: 'Colab Git Push'")
print("  3. Select scope: 'repo' (check the box)")
print("  4. Click 'Generate token'")
print("  5. Copy the token (you won't see it again!)")
print("=" * 60)

git_username = input("Enter your GitHub username: ")
git_token = getpass("Enter your GitHub Personal Access Token: ")
git_email = input("Enter your Git email: ")

# Configure git
!git config --global user.name "{git_username}"
!git config --global user.email "{git_email}"

# Create authenticated URL
GIT_REPO_URL_AUTH = GIT_REPO_URL.replace(
    "https://",
    f"https://{git_username}:{git_token}@"
)

print("\n✓ Git credentials configured")
print(f"  Username: {git_username}")
print(f"  Email: {git_email}")

In [ ]:
# CELL 6: Clone Repository

# Remove repo folder if exists
if os.path.exists(REPO_FOLDER):
    shutil.rmtree(REPO_FOLDER)

# Clone the repository
print(f"Cloning repository...")
!git clone {GIT_REPO_URL_AUTH} {REPO_FOLDER}

# Change to repo directory and checkout branch
%cd {REPO_FOLDER}

print(f"\nChecking out branch: {GIT_BRANCH}")
!git checkout {GIT_BRANCH}

print("\n✓ Repository cloned and branch checked out")

In [ ]:
# CELL 7: Copy Downloaded Files to Repository

target_path = os.path.join(REPO_FOLDER, TARGET_FOLDER)
print(f"Copying files to: {TARGET_FOLDER}/")

# Create target directory
os.makedirs(target_path, exist_ok=True)

# Copy all downloaded files
copied_count = 0
for item in os.listdir(DOWNLOAD_FOLDER):
    source = os.path.join(DOWNLOAD_FOLDER, item)
    destination = os.path.join(target_path, item)
    
    if os.path.isfile(source):
        shutil.copy2(source, destination)
        print(f"  ✓ Copied file: {item}")
        copied_count += 1
    elif os.path.isdir(source):
        if os.path.exists(destination):
            shutil.rmtree(destination)
        shutil.copytree(source, destination)
        print(f"  ✓ Copied folder: {item}")
        copied_count += 1

print(f"\n✓ Copied {copied_count} items to repository")

# Show git status
print("\nGit status:")
%cd {REPO_FOLDER}
!git status

In [ ]:
# CELL 8: Commit and Push Changes

%cd {REPO_FOLDER}

# Add all new files
print("Adding files to git...")
!git add .

# Show what will be committed
print("\nFiles to be committed:")
!git status --short

# Commit message
commit_message = f"""Add files from Google Drive folder

Downloaded from: {GDRIVE_FOLDER_URL}
Target location: {TARGET_FOLDER}/
Downloaded via Google Colab
"""

# Commit the changes
print("\nCommitting changes...")
!git commit -m "{commit_message}"

# Push to the branch
print(f"\nPushing to branch: {GIT_BRANCH}...")
!git push -u origin {GIT_BRANCH}

print("\n" + "=" * 60)
print("✓ SUCCESS! Files have been pushed to GitHub")
print("=" * 60)
print(f"\nRepository: {GIT_REPO_URL}")
print(f"Branch: {GIT_BRANCH}")
print(f"Files location: {TARGET_FOLDER}/")
print(f"\nView at: {GIT_REPO_URL.replace('.git', '')}/tree/{GIT_BRANCH}/{TARGET_FOLDER}")

In [ ]:
# CELL 9 (Optional): Verify the Push

%cd {REPO_FOLDER}

print("Latest commit:")
!git log -1 --stat

print("\nRemote branches:")
!git branch -r

print("\n✓ All done! Check your GitHub repository to see the files.")